In [ ]:
# ===============================
# Stap 0: Bibliotheken installeren
# ===============================
!pip install lime shap imgkit --quiet
!apt-get install -y wkhtmltopdf > /dev/null

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import lime
import lime.lime_tabular
import shap
import imgkit
from IPython.display import display, Markdown

# ===============================
# Stap 1: Dataset laden
# ===============================
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
df.head()

# ===============================
# Stap 2: Data voorbereiden
# ===============================
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
target = 'Survived'

# Vul ontbrekende waarden
df['Age'].fillna(df['Age'].median(), inplace=True)

# Zet geslacht om naar nummers
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])

# Split in train en test set
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ===============================
# Stap 3: Model trainen
# ===============================
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print("Model accuracy:", model.score(X_test, y_test))

# ===============================
# Stap 4: LIME uitleg genereren
# ===============================
explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=X_train.columns,
    class_names=['Not Survived','Survived'],
    mode='classification'
)

i = 0  # eerste testvoorbeeld
exp = explainer_lime.explain_instance(X_test.values[i], model.predict_proba)

# Sla LIME HTML op
exp.save_to_file("lime_example.html")

# Converteer LIME HTML automatisch naar PNG
imgkit.from_file('lime_example.html', 'lime_example.png')
print("LIME uitleg opgeslagen en geconverteerd naar lime_example.png")

# ===============================
# Stap 5: SHAP uitleg genereren
# ===============================
explainer_shap = shap.LinearExplainer(model, X_train, feature_perturbation="interventional")
shap_values = explainer_shap.shap_values(X_test)

# Plot SHAP summary en sla op als PNG
shap.summary_plot(shap_values, X_test, show=False)
plt.savefig("shap_summary.png", bbox_inches='tight')
plt.close()
print("SHAP summary plot opgeslagen als shap_summary.png")

# ===============================
# Stap 6: Markdown notities GDPR + visualisaties
# ===============================
markdown_text = """
# Week 7 – Transparantie en GDPR Observaties

**Doel van deze notities:**
- Begrijpen hoe AI-modellen transparant kunnen zijn
- GDPR basisprincipes toepassen op datasets en modellen

## 1. GDPR Basisprincipes
- Recht op inzage
- Recht op correctie
- Data minimalisatie
- Transparantie
- Toestemming

## 2. Analyse van dataset en model
- Dataset: Titanic (Passenger data)
- Features: Pclass, Sex, Age, SibSp, Parch
- Target: Survived (1=overleefd, 0=overleden)

**Observaties:**
- Geen echte persoonsgegevens
- Kolommen zoals Name, Ticket, Cabin zouden PII kunnen bevatten
- Model transparantie verbeterd met LIME en SHAP

## 3. Explainable AI Observaties
- LIME voorbeeld: ![LIME Example](lime_example.png)
- SHAP summary plot: ![SHAP Summary](shap_summary.png)

## 4. Privacy & Transparantie Verbeteringen
1. Anonimiseer persoonlijke informatie (namen, e-mail, telefoonnummers)
2. Gebruik alleen noodzakelijke features voor modeltraining
3. Documenteer alle stappen in Markdown of Notion
4. Bied uitleg aan eindgebruikers (LIME/SHAP)
5. Controleer websites of data-bronnen op security issues (Mozilla Observatory)

## 5. Conclusie
- Model transparanter en uitlegbaar met LIME & SHAP
- GDPR principes deels toegepast
- Volgende stap: combineren van AI uitleg met security testen voor volledige compliance
"""

display(Markdown(markdown_text))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.8 MB/s eta 0:00:00
Extracting templates from packages: 100%
